In [1]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

with open("myenv.yml", "w") as f:
    f.write(myenv.serialize_to_string())

In [1]:
from azureml.core.image import ContainerImage
from azureml.core.image import Image
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'mlp_classifier')

# configure the image
image_config = ContainerImage.image_configuration(runtime="python",
                                                  execution_script="score.py",
                                                  conda_file="myenv.yml")

# create the image
image = Image.create(name='mlp-classifier-img', models=[model], image_config=image_config, workspace=ws)

# wait for image creation to finish
image.wait_for_creation(show_output=True)

Creating image
Running...............................
Succeeded
Image creation operation finished for image mlp-classifier-img:2, operation "Succeeded"


In [3]:
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice

# configure an ACI-based deployment
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2)

aci_service = Webservice.deploy_from_image(deployment_config=aci_config, 
                                           image=image, 
                                           name='mlp-classifier-svc', 
                                           workspace=ws)
aci_service.wait_for_deployment(show_output=True)    

Creating service
Running................
SucceededACI service creation operation finished, operation "Succeeded"


In [4]:
aci_service.scoring_uri

'http://0e65851a-ff05-4f80-b105-e2a089668fa5.centralus.azurecontainer.io/score'